#### a. Descargar el Quijote
https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79

In [2]:
// !wget "https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt"
val df = spark.read.option("header", "false").csv("el_quijote.txt")

df: org.apache.spark.sql.DataFrame = [_c0: string]


Aplicar no solo count (para obtener el número de líneas) y show sino probar distintas
sobrecargas del método show (con/sin truncate, indicando/sin indicar num de filas,
etc) así como también los métodos, head, take, first (diferencias entre estos 3?)

In [3]:
df.count()

res0: Long = 2185


In [5]:
df.show(3,truncate = false)

+----------------------------+
|_c0                         |
+----------------------------+
|DON QUIJOTE DE LA MANCHA    |
|Miguel de Cervantes Saavedra|
|PRIMERA PARTE               |
+----------------------------+
only showing top 3 rows



In [6]:
df.head(3)

res3: Array[org.apache.spark.sql.Row] = Array([DON QUIJOTE DE LA MANCHA], [Miguel de Cervantes Saavedra], [PRIMERA PARTE])


In [7]:
df.take(3)

res4: Array[org.apache.spark.sql.Row] = Array([DON QUIJOTE DE LA MANCHA], [Miguel de Cervantes Saavedra], [PRIMERA PARTE])


In [8]:
df.first()

res5: org.apache.spark.sql.Row = [DON QUIJOTE DE LA MANCHA]


#### b. Del ejercicio de M&M aplicar:

In [10]:
val mnm_file = "mnm_dataset.csv"
val mnm_df = spark
            .read
            .option("header", "true")
            .option("inferSchema", "true")
            .csv(mnm_file)
mnm_df.show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   10|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



mnm_file: String = mnm_dataset.csv
mnm_df: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 1 more field]


i. Otras operaciones de agregación como el Max con otro tipo de ordenamiento (descendiente).

In [19]:
//import spark.sql.functions 
val max_mnm_df = (mnm_df
                    .select("State", "Color", "Count")
                    .groupBy("State", "Color")
                    .agg(max("Count").alias("Maximo"))
                    .orderBy(desc("Maximo")))
max_mnm_df.show(5)

+-----+------+------+
|State| Color|Maximo|
+-----+------+------+
|   CO|  Blue|   100|
|   NM|Orange|   100|
|   UT|  Blue|   100|
|   WA|Orange|   100|
|   WY| Green|   100|
+-----+------+------+
only showing top 5 rows



max_mnm_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


ii. hacer un ejercicio como el “where” de CA que aparece en el libro pero
indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [24]:
val nv_co_count_mnm_df = (mnm_df
                             .select("State", "Color", "Count")
                             .where((col("State") === "NV") or (col("State") === "CO"))
                             .groupBy("State", "Color")
                             .agg(count("Count").alias("Total"))
                             .orderBy(desc("Total")))
nv_co_count_mnm_df.show()

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CO|Yellow| 1721|
|   CO| Green| 1713|
|   NV|Orange| 1712|
|   NV| Green| 1698|
|   CO|  Blue| 1695|
|   NV|Yellow| 1675|
|   NV|  Blue| 1673|
|   NV| Brown| 1657|
|   CO| Brown| 1656|
|   CO|Orange| 1642|
|   CO|   Red| 1624|
|   NV|   Red| 1610|
+-----+------+-----+



nv_co_count_mnm_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [State: string, Color: string ... 1 more field]


iii. Hacer un ejercicio donde se calculen en una misma operación el Max, Min, Avg, Count. Revisar el API (documentación) donde encontrarán este ejemplo: ds.agg(max("𝑎𝑔𝑒"),𝑎𝑣𝑔("salary")) ds.groupBy().agg(max("𝑎𝑔𝑒"),𝑎𝑣𝑔("salary")) NOTA: $ es un alias de col()

In [25]:
val op_mnm_df = (mnm_df
                   .select("State", "Color", "Count")
                   .groupBy("State", "Color")
                   .agg(
                         count("Count").alias("Total"),
                         max("Count").alias("Max"),
                         min("Count").alias("Min"),
                         round(avg("Count"),3).alias("Avg")
                       )
                )
op_mnm_df.show(5)

+-----+------+-----+---+---+------+
|State| Color|Total|Max|Min|   Avg|
+-----+------+-----+---+---+------+
|   WY| Green| 1695|100| 10|55.657|
|   NV|   Red| 1610|100| 10|55.494|
|   UT|  Blue| 1655|100| 10|54.367|
|   WA|Orange| 1658|100| 10|  55.2|
|   NM| Green| 1682|100| 10|54.197|
+-----+------+-----+---+---+------+
only showing top 5 rows



op_mnm_df: org.apache.spark.sql.DataFrame = [State: string, Color: string ... 4 more fields]


iv. Hacer también ejercicios en SQL creando tmpView

In [30]:
mnm_df.createOrReplaceTempView("mnm")
spark.sql("""SELECT * FROM mnm""").show(5)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   10|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
+-----+------+-----+
only showing top 5 rows



In [31]:
spark.sql("""SELECT State,
                    Color,
                    count(Count) as Total,
                    max(Count) as Maximo,
                    min(Count) as Minimo,
                    round(avg(Count),3) as Avg
             FROM mnm
             GROUP BY State,Color
              """).show()

+-----+------+-----+------+------+------+
|State| Color|Total|Maximo|Minimo|   Avg|
+-----+------+-----+------+------+------+
|   WY| Green| 1695|   100|    10|55.657|
|   NV|   Red| 1610|   100|    10|55.494|
|   UT|  Blue| 1655|   100|    10|54.367|
|   WA|Orange| 1658|   100|    10|  55.2|
|   NM| Green| 1682|   100|    10|54.197|
|   CA|  Blue| 1603|   100|    10|55.598|
|   WA|   Red| 1671|   100|    10|55.854|
|   NV| Brown| 1657|   100|    10|55.811|
|   AZ| Green| 1676|   100|    10|54.822|
|   CA|   Red| 1656|   100|    10| 55.27|
|   AZ|Orange| 1689|   100|    10|54.283|
|   CO|  Blue| 1695|   100|    10| 55.11|
|   NM|Orange| 1665|   100|    10|54.805|
|   NM|Yellow| 1688|   100|    10|54.945|
|   WY|Orange| 1595|   100|    10|55.145|
|   UT|Yellow| 1645|   100|    10|54.264|
|   WY|   Red| 1670|   100|    10|54.951|
|   OR|  Blue| 1646|   100|    10|54.998|
|   NV|Orange| 1712|   100|    10|54.865|
|   AZ|Yellow| 1654|   100|    10|54.985|
+-----+------+-----+------+------+

In [ ]:
spark.sql("""SELECT State,Color,COUNT(*) as Cantidad
             FROM mnm  
             WHERE State = "TX" 
             GROUP BY State,Color
             ORDER BY Cantidad DESC
             """).show()